<a href="https://colab.research.google.com/github/eirik-dn/NLP-For-Finance-Final-Assignment/blob/main/NLP_For_Finance_Final_Assignment_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# NLP For Finance Final Assignment
# LLM Code

**Name: Eirik Dalen Nyrén**

**Student ID:102060**

---

In [1]:

!pip install transformers datasets[torch] nltk nlpaug scikit-learn torch seaborn pdfplumber pdfminer.six linearmodels swifter
!pip uninstall -y numpy gensim scipy smart-open
!pip install numpy==1.25.2 scipy gensim smart-open
!pip install tdqm
!apt install -y nvidia-cuda-toolkit
!pip install llama-cpp-python'==0.2.53' --force-reinstall --no-cache-dir
!pip install huggingface_hub[hf_xet]


Found existing installation: numpy 1.25.2
Uninstalling numpy-1.25.2:
  Successfully uninstalled numpy-1.25.2
Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
Found existing installation: scipy 1.13.1
Uninstalling scipy-1.13.1:
  Successfully uninstalled scipy-1.13.1
Found existing installation: smart-open 7.1.0
Uninstalling smart-open-7.1.0:
  Successfully uninstalled smart-open-7.1.0
  Using cached numpy-1.25.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached smart_open-7.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-1.25.2-cp311-cp311-manylinux_2_17_x86_64.manylinu

^C
^C
^C


In [1]:
# import packages

# Core packages
import os
import re
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns

# NLTK for text preprocessing
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.sentiment.util import mark_negation
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import ngrams
# function for preproscessing
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
nltk.download("wordnet")

# spacy for text preprocessing
import spacy

# Sklearn for vectorization, similarity metrics, clustering
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

# Gensim for word embeddings
import gensim
import gensim.downloader as api
from gensim.models import Word2Vec, KeyedVectors, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.utils import simple_preprocess

# Statsmodels for statistical modeling
import statsmodels.api as sm
import statsmodels.formula.api as smf

# swifter
import swifter

# import tdqm to look at progression of loops
from tqdm import tqdm
tqdm.pandas()

# import sql
import sqlite3
conn = sqlite3.connect('LLMs_results.db')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# @title Load the textual data for the Earnings Calls of S&P500 from 2015 to 2021
import pandas as pd

# Load the Sample of Earnings Calls for the S&P500 from 2015 to 2021 with financials
Sample_EC = pd.read_csv("https://www.dropbox.com/scl/fi/2p7ahxroqj9pwf98ni5an/Sample_Calls.csv?rlkey=zfieicvz891u4e3z0aroeg0u7&dl=1")

# Load the Sample's Presentation texts
Sample_Presentations = pd.read_feather("https://www.dropbox.com/scl/fi/uceh2xva5g4apbmt92cgt/Sample_Calls_Presentations.feather?rlkey=ln4nzsa4nenqyvm0pg2cur9sp&dl=1")

# Load the Q&A session textual data for the sample
Sample_QAs = pd.read_feather("https://www.dropbox.com/scl/fi/iq4111nlmsykp2tzxk9xg/Sample_Calls_QA.feather?rlkey=xabjqmwhesx05jivrlfzkgj6m&dl=1")


## Note: column "file_name" is to be used for the merging

# ## Columns:
#     # Identifiers
#     "GVKEY": "A unique company identifier used by Compustat.",
#     "date_rdq": "The reporting date of the quarterly earnings or a related key event date.",
#     "co_conm": "The company’s name in CRSP.",

#     # Earnings Call Columns
#     "file_name": "The identifier or filename of the earnings call transcript.",
#     "CAR-11-Carhart": "Cumulative Abnormal Return over an event window using the Carhart 4-factor model.",
#     "CAR-11-ff3": "Cumulative Abnormal Return over an event window using the Fama-French 3-factor model.",
#     "CAR01-Carhart": "Cumulative Abnormal Return (alternative window) using the Carhart 4-factor model.",
#     "CAR01-ff3": "Cumulative Abnormal Return (alternative window) using the Fama-French 3-factor model.",
#     "IV": "Implied volatility (often from options) reflecting expected future stock price volatility.",
#     "hvol": "Historical volatility of the stock, based on past price movements.",
#     "IV_l1d": "Implied volatility lagged by one day.",
#     "IV_l2d": "Implied volatility lagged by two days.",
#     "IV_f1d": "Implied volatility forecasted or measured one day forward.",

#     # I/B/E/S Columns
#     "NUMEST": "The number of analyst estimates contributing to the consensus.",
#     "NUMUP": "The number of analysts who have revised their EPS estimates upward.",
#     "NUMDOWN": "The number of analysts who have revised their EPS estimates downward.",
#     "MEDEST": "The median of analyst EPS estimates.",
#     "MEANEST": "The mean of analyst EPS estimates.",
#     "ACTUAL": "The I/B/E/S standardized actual EPS figure, often adjusted for comparability.",
#     "surp": "The earnings surprise, typically ACTUAL minus MEANEST.",
#     "SurpDec": "A scaled or decimalized version of the earnings surprise.",

#     # Compustat Columns
#     "atq": "Total Assets (Quarterly)",
#     "actq": "Current Assets (Quarterly)",
#     "cheq": "Cash and Cash Equivalents (Quarterly)",
#     "rectq": "Accounts Receivable (Quarterly)",
#     "invtq": "Inventory (Quarterly)",
#     "ltq": "Total Liabilities (Quarterly)",
#     "lctq": "Current Liabilities (Quarterly)",
#     "apq": "Accounts Payable (Quarterly)",
#     "ceqq": "Total Equity (Quarterly)",
#     "seqq": "Common Equity (Quarterly)",

#     "capxy": "Capital Expenditures (Note: 'capxy' is annual by default, quarterly approximations derived from segments)",
#     "dpq": "Depreciation and Amortization (Quarterly)",
#     "saleq": "Revenue (Quarterly)",
#     "cogsq": "Cost of Goods Sold (Quarterly)",
#     "oiadpq": "Operating Income (Quarterly)",
#     "niq": "Net Income (Quarterly)",
#     "epspxq": "Basic Earnings Per Share (Quarterly)",
#     "epspiq": "Diluted Earnings Per Share (Quarterly)",
#     "dlttq": "Long-Term Debt (Quarterly)",
#     "dlcq": "Debt in Current Liabilities (Quarterly)",
#     "prccq": "Price Close - Fiscal Quarter",
#     "cshoq": "Common Shares Outstanding (Quarterly)",
#     "dvpq": "Dividends Paid (Quarterly)",
#     "xintq": "Interest Expense (Quarterly)"


In [3]:
# import the Loughran-McDonald_MasterDictionary
github_LM_dict_url = "https://raw.githubusercontent.com/eirik-dn/NLP-For-Finance-Final-Assignment/refs/heads/main/Loughran-McDonald_MasterDictionary_1993-2024.csv"
LM_dict_global            = pd.read_csv(github_LM_dict_url)
LM_dict_global.head()

,Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Complexity,Syllables,Source
0,AARDVARK,1,755,2.955070e-08,1.945421e-08,4.078069e-06,140,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.174200e-10,8.060019e-12,8.919011e-09,1,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,9,3.522600e-10,1.089343e-10,5.105359e-08,7,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,29,1.135060e-09,6.197922e-10,1.539279e-07,28,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,9620,3.765268e-07,3.825261e-07,3.421836e-05,1295,0,0,0,0,0,0,0,0,3,12of12inf


In [4]:
# import the harvard dictonary¨
HV=pd.read_csv("https://www.dropbox.com/s/wjucnpw39uuxupf/HarvardGI4.txt?dl=1", delimiter='\t',low_memory=False)
HV.head()


,Entry,Source,Positiv,Negativ,Pstv,Affil,Ngtv,Hostile,Strong,Power,...,Anomie,NegAff,PosAff,SureLw,If,NotLw,TimeSpc,FormLw,Othtags,Defined\
0,A,H4Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET ART,| article: Indefinite singular article--some o...
1,ABANDON,H4Lvd,NaN,Negativ,NaN,NaN,Ngtv,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|\
2,ABANDONMENT,H4,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,|\
3,ABATE,H4Lvd,NaN,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUPV,|\
4,ABATEMENT,Lvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Noun,\


In [5]:
# extract the answers from Sample_QA
temp = Sample_QAs[Sample_QAs['QA'] == "a"]

# Generate Earnings call level data by aggregating in the file_name level
temp = temp.groupby('file_name')['QA_text'].apply(lambda x: ''.join(x)).reset_index()

# Merge with Sample_EC and name it as Sample_Answers
Sample_answers_global = Sample_EC.merge(temp)
Sample_answers_global.head()

,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,niq,epspxq,epspiq,dlttq,dlcq,prccq,cshoq,dvpq,xintq,QA_text
0,16101.0,2016-07-29 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.011886,0.014261,0.014261,0.021246,0.179151,0.129186,...,1610.0,0.99,0.99,37328.0,517.0,61.91,1628.542,0.0,245.0,"Jeff, this is Rick. I will cover the first and..."
1,16101.0,2016-04-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.026387,0.023499,0.023499,0.021770,0.289777,0.114447,...,1354.0,0.83,0.83,29490.0,2423.0,57.12,1617.359,0.0,215.0,"Okay. Hi, Jeff, it's Rick. So I'll take I gues..."
2,16101.0,2016-10-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.078668,-0.079290,-0.079290,-0.092594,0.253269,0.381002,...,1598.0,0.97,0.97,37284.0,26.0,63.07,1624.908,0.0,271.0,"Sure, Jami; this is Rick. Thank you for the qu..."
3,16101.0,2017-01-27 14:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,-0.010152,-0.000737,-0.000737,-0.005279,0.182080,0.145941,...,1391.0,0.85,0.85,36440.0,402.0,62.62,1592.513,0.0,277.0,"Hi, Jami; it's Bill. So on your operating marg..."
4,16101.0,2017-04-27 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,0.010397,0.010672,0.010672,0.012819,0.192822,0.112189,...,1711.0,1.07,1.07,36526.0,425.0,65.16,1591.366,0.0,273.0,"Yes. Well, first, thanks for the question. May..."


In [6]:
# Merge EC and presentations
EC_pres_global = pd.merge(
    Sample_EC,
    Sample_Presentations[['file_name','presentation']],
    on = 'file_name',
    how = 'outer'
)

EC_pres_global.head()

,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,niq,epspxq,epspiq,dlttq,dlcq,prccq,cshoq,dvpq,xintq,presentation
0,143356.0,2016-05-05 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.015353,-0.012682,-0.012682,-0.007504,0.259517,0.180580,...,1336.0,2.97,2.97,22554.0,9326.0,72.22,443.146,0.0,325.0,"Ladies and gentlemen, thank you for standi..."
1,143356.0,2016-08-04 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.000416,-0.000815,-0.000815,-0.017714,0.239821,0.421283,...,921.0,2.06,2.06,21080.0,8011.0,71.34,438.739,0.0,357.0,Welcome to the Prudential quarterly earnin...
2,143356.0,2016-11-03 13:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.031476,0.030910,0.030910,0.024372,0.277495,0.228244,...,1827.0,4.14,4.14,21480.0,7737.0,81.65,431.704,0.0,314.0,"Ladies and gentlemen, thank you for standi..."
3,6730.0,2016-04-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.004689,-0.003578,-0.003578,-0.003950,0.213235,0.190130,...,440.1,0.42,0.42,7477.6,648.3,72.01,1103.837,0.0,56.8,"Ladies and gentlemen, thank you for standi..."
4,6730.0,2016-07-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.027560,0.027598,0.027598,0.021945,0.186548,0.109124,...,747.7,0.71,0.71,8685.5,645.8,78.75,1103.843,0.0,57.2,"Ladies and gentlemen, thank you for standi..."


# **Part 2: Large Lagugage Models (LLMs)**



**Approach 1: FinBert**

**Goal: Do sentiment analysis using FinBert**

**Steps:**


*   Step 1: Import needed packages
*   Step 2 Make sure I use GPU and find FinBert from Huggingface
*   Step 3: Performe chunking by sentences and uses overlapping  
to make sure that FinBert does not classify based on not full sentences.
*   Step 4:Use the FinBert to calculate sentiment score as P(pos) - P(neg) and standardize the scores

In [7]:
# create initial DF's
EC_pres_llm     = EC_pres_global.copy()
answers_llm     = Sample_answers_global.copy()

In [8]:
# import packages
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoConfig
import torch.nn.functional as F


In [9]:
# check if gpu is avaliable
if torch.cuda.is_available():
    print("GPU is available.")
    device = torch.device("cuda")
else:
    print("GPU is not available, using CPU instead.")
    device = torch.device("cpu")

GPU is available.


In [10]:
# import and load Finbert (Prosus AI) fron huggingface
model_name   = 'ProsusAI/finbert'

# use the tokenizer
tokenizer     = AutoTokenizer.from_pretrained(model_name)

# use the configuration
config        = AutoConfig.from_pretrained(model_name, output_hidden_states = True)

# use the model
model_finbert = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
# performe simple test of the model
sample_text = "The market outlook is positive."

inputs = tokenizer(sample_text, return_tensors="pt")

outputs = model_finbert(**inputs)

#print(outputs)


**Step 3: Performe chunking by sentences and uses overlapping  
to make sure that FinBert does not classify based on not full sentences.**

In [12]:
def chunk_text_by_sentences_with_overlap(text, tokenizer, max_length=450, overlap_sentences=2, metadata=None):
    """
    Split text into chunks by sentences with overlap, optionally adding metadata to each chunk.

    Args:
        text (str): The input text to chunk.
        tokenizer: The tokenizer to use for token counting.
        max_length (int): Maximum number of tokens per chunk (default: 450, ~1,800 chars).
        overlap_sentences (int): Number of sentences to overlap between chunks (default: 2).
        metadata (dict): Optional metadata to prepend to each chunk (e.g., {"date": "2023-01-15", "section": "Presentation"}).

    Returns:
        list: List of chunked texts (with metadata prepended if provided).
    """
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk_sents = []
    current_tokens_count = 0

    for idx, sent in enumerate(sentences):
        sent_tokens = tokenizer.encode(sent, add_special_tokens=True)
        sent_len = len(sent_tokens) - 2  # Subtract [CLS] and [SEP]

        if current_tokens_count + sent_len > (max_length - 2):
            chunk_text = "".join(current_chunk_sents)
            # Prepend metadata if provided
            if metadata:
                metadata_str = f"Date: {metadata.get('date', 'Unknown')}, Section: {metadata.get('section', 'Unknown')}\n"
                chunk_text = metadata_str + chunk_text
            chunks.append(chunk_text)

            overlap_start_idx = max(0, len(current_chunk_sents) - overlap_sentences)
            overlap_sents = current_chunk_sents[overlap_start_idx:]
            current_chunk_sents = overlap_sents.copy()
            current_tokens_count = sum(
                len(tokenizer.encode(s, add_special_tokens=False)) for s in current_chunk_sents
            )

        current_chunk_sents.append(sent)
        current_tokens_count += sent_len

    if current_chunk_sents:
        chunk_text = "".join(current_chunk_sents)
        if metadata:
            metadata_str = f"Date: {metadata.get('date', 'Unknown')}, Section: {metadata.get('section', 'Unknown')}\n"
            chunk_text = metadata_str + chunk_text
        chunks.append(chunk_text)

    return chunks

In [13]:
def chunk_text_by_sentences_with_overlap(text, tokenizer, max_length=450, overlap_sentences=2, metadata=None):
    """
    Split text into chunks by sentences with overlap, optionally adding metadata to each chunk.

    Args:
        text (str): The input text to chunk.
        tokenizer: The tokenizer to use for token counting.
        max_length (int): Maximum number of tokens per chunk (default: 450, ~1,800 chars).
        overlap_sentences (int): Number of sentences to overlap between chunks (default: 2).
        metadata (dict): Optional metadata to prepend to each chunk (e.g., {"date": "2023-01-15", "section": "Presentation"}).

    Returns:
        list: List of chunked texts (with metadata prepended if provided).
    """
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk_sents = []
    current_tokens_count = 0

    for idx, sent in enumerate(sentences):
        sent_tokens = tokenizer.encode(sent, add_special_tokens=True)
        sent_len = len(sent_tokens) - 2  # Subtract [CLS] and [SEP]

        if current_tokens_count + sent_len > (max_length - 2):
            chunk_text = "".join(current_chunk_sents)
            # Prepend metadata if provided
            if metadata:
                metadata_str = f"Date: {metadata.get('date', 'Unknown')}, Section: {metadata.get('section', 'Unknown')}\n"
                chunk_text = metadata_str + chunk_text
            chunks.append(chunk_text)

            overlap_start_idx = max(0, len(current_chunk_sents) - overlap_sentences)
            overlap_sents = current_chunk_sents[overlap_start_idx:]
            current_chunk_sents = overlap_sents.copy()
            current_tokens_count = sum(
                len(tokenizer.encode(s, add_special_tokens=False)) for s in current_chunk_sents
            )

        current_chunk_sents.append(sent)
        current_tokens_count += sent_len

    if current_chunk_sents:
        chunk_text = "".join(current_chunk_sents)
        if metadata:
            metadata_str = f"Date: {metadata.get('date', 'Unknown')}, Section: {metadata.get('section', 'Unknown')}\n"
            chunk_text = metadata_str + chunk_text
        chunks.append(chunk_text)

    return chunks

In [14]:
def analyze_long_text_sentiment(text, tokenizer, model, max_length=450, overlap_sentences=1, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Ensure model is on the correct device
    model.to(device)

    # Chunk text
    chunks = chunk_text_by_sentences_with_overlap(
        text=text,
        tokenizer=tokenizer,
        max_length=max_length,
        overlap_sentences=overlap_sentences
    )

    label_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}
    weighted_probs = None
    total_tokens = 0
    chunk_labels = []

    # Classify each chunk
    for chunk_text in chunks:
        tokens = tokenizer.encode(chunk_text, add_special_tokens=False)
        token_count = len(tokens)

        inputs = tokenizer(chunk_text, return_tensors="pt", truncation=True, max_length=max_length)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits
        probs = F.softmax(logits, dim=1)
        avg_probs = probs.mean(dim=0)

        if weighted_probs is None:
            weighted_probs = avg_probs * token_count
        else:
            weighted_probs += avg_probs * token_count
        total_tokens += token_count

        pred_label_id = torch.argmax(avg_probs).item()
        chunk_labels.append(label_mapping[pred_label_id])

    # Aggregate
    if total_tokens > 0:
        final_probs = weighted_probs / total_tokens
        final_label_id = torch.argmax(final_probs).item()
        final_label = label_mapping[final_label_id]
    else:
        final_label = "Neutral"
        final_probs = torch.tensor([0.0, 1.0, 0.0], device=device)

    # Handle CPU/GPU for numpy conversion
    final_probs = final_probs.numpy() if device.type == "cpu" else final_probs.cpu().numpy()
    return final_label, final_probs, chunk_labels

In [15]:
# create dataframes to use for finbert from the global dataframes
#EC_pres_FinBert     = EC_pres_global.copy()
#EC_answers_FinBert  = Sample_answers_global.copy()

In [16]:
# Create empty lists to store results for presentations
final_labels_pres_finbert     = []
probabilities_pres_finbert    = []
all_chunk_labels_pres_finbert = []

# Process each row with progress bar
for idx, row in tqdm(EC_pres_llm.iterrows(), total=EC_pres_llm.shape[0], desc="Processing rows"):
    text = row["presentation"]
    # Call the sentiment analysis function and unpack the results
    final_label, final_probs, chunk_labels = analyze_long_text_sentiment(
        text,
        tokenizer,
        model_finbert,
        max_length=450,
        overlap_sentences=2,
        device=device,
        )

    # Append each result to the corresponding list
    final_labels_pres_finbert.append(final_label)
    probabilities_pres_finbert.append(final_probs)
    all_chunk_labels_pres_finbert.append(chunk_labels)

# Add new columns to the DataFrame
EC_pres_llm["final_sentiment_pres_finbert"]     = final_labels_pres_finbert
EC_pres_llm["final_probabilities_pres_finbert"] = probabilities_pres_finbert  # array of [neg, neu, pos]
EC_pres_llm["chunk_labels_pres_finbert"]        = all_chunk_labels_pres_finbert

# Display the DataFrame
EC_pres_llm.head()

Processing rows: 100%|██████████| 2877/2877 [08:22<00:00,  5.73it/s]


,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,dlttq,dlcq,prccq,cshoq,dvpq,xintq,presentation,final_sentiment_pres_finbert,final_probabilities_pres_finbert,chunk_labels_pres_finbert
0,143356.0,2016-05-05 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.015353,-0.012682,-0.012682,-0.007504,0.259517,0.180580,...,22554.0,9326.0,72.22,443.146,0.0,325.0,"Ladies and gentlemen, thank you for standi...",Neutral,"[0.37680644, 0.50930405, 0.113889486]","[Neutral, Negative, Negative, Negative, Neutra..."
1,143356.0,2016-08-04 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.000416,-0.000815,-0.000815,-0.017714,0.239821,0.421283,...,21080.0,8011.0,71.34,438.739,0.0,357.0,Welcome to the Prudential quarterly earnin...,Negative,"[0.45286137, 0.33879948, 0.20833908]","[Positive, Negative, Negative, Negative, Neutr..."
2,143356.0,2016-11-03 13:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.031476,0.030910,0.030910,0.024372,0.277495,0.228244,...,21480.0,7737.0,81.65,431.704,0.0,314.0,"Ladies and gentlemen, thank you for standi...",Negative,"[0.59855324, 0.21789162, 0.18355516]","[Negative, Negative, Negative, Negative, Negat..."
3,6730.0,2016-04-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.004689,-0.003578,-0.003578,-0.003950,0.213235,0.190130,...,7477.6,648.3,72.01,1103.837,0.0,56.8,"Ladies and gentlemen, thank you for standi...",Negative,"[0.43463686, 0.30363905, 0.26172405]","[Positive, Negative, Negative, Positive, Posit..."
4,6730.0,2016-07-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.027560,0.027598,0.027598,0.021945,0.186548,0.109124,...,8685.5,645.8,78.75,1103.843,0.0,57.2,"Ladies and gentlemen, thank you for standi...",Negative,"[0.6555745, 0.10334945, 0.24107622]","[Positive, Negative, Negative, Negative, Negat..."


In [17]:
# Create empty lists to store results
final_labels_answers_finbert     = []
probabilities_answers_finbert    = []
all_chunk_labels_answers_finbert = []

# process the functions and get sentinement scores for answers
for idx, row in tqdm(answers_llm.iterrows(), total=answers_llm.shape[0], desc="Processing answers"):
    text = row["QA_text"]
    final_label, final_probs, chunk_labels = analyze_long_text_sentiment(
        text,
        tokenizer,
        model_finbert,
        max_length=450,
        overlap_sentences=2,
        device=device
    )
    final_labels_answers_finbert.append(final_label)
    probabilities_answers_finbert.append(final_probs)
    all_chunk_labels_answers_finbert.append(chunk_labels)

# Add to DataFrame
answers_llm["final_sentiment_answers_finbert"] = final_labels_answers_finbert
answers_llm["final_probabilities_answers_finbert"] = probabilities_answers_finbert
answers_llm["chunk_labels_answers_finbert"] = all_chunk_labels_answers_finbert

# Display the DataFrame
answers_llm.head()

Processing answers: 100%|██████████| 2877/2877 [10:24<00:00,  4.61it/s]


,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,dlttq,dlcq,prccq,cshoq,dvpq,xintq,QA_text,final_sentiment_answers_finbert,final_probabilities_answers_finbert,chunk_labels_answers_finbert
0,16101.0,2016-07-29 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.011886,0.014261,0.014261,0.021246,0.179151,0.129186,...,37328.0,517.0,61.91,1628.542,0.0,245.0,"Jeff, this is Rick. I will cover the first and...",Positive,"[0.35596988, 0.05079206, 0.593238]","[Negative, Positive, Positive, Negative, Negat..."
1,16101.0,2016-04-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.026387,0.023499,0.023499,0.021770,0.289777,0.114447,...,29490.0,2423.0,57.12,1617.359,0.0,215.0,"Okay. Hi, Jeff, it's Rick. So I'll take I gues...",Positive,"[0.21798027, 0.027822783, 0.7541969]","[Positive, Positive, Negative, Positive, Posit..."
2,16101.0,2016-10-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.078668,-0.079290,-0.079290,-0.092594,0.253269,0.381002,...,37284.0,26.0,63.07,1624.908,0.0,271.0,"Sure, Jami; this is Rick. Thank you for the qu...",Positive,"[0.28801477, 0.061897594, 0.6500877]","[Positive, Positive, Positive, Negative, Posit..."
3,16101.0,2017-01-27 14:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,-0.010152,-0.000737,-0.000737,-0.005279,0.182080,0.145941,...,36440.0,402.0,62.62,1592.513,0.0,277.0,"Hi, Jami; it's Bill. So on your operating marg...",Positive,"[0.22084142, 0.030022986, 0.74913555]","[Negative, Positive, Positive, Positive, Negat..."
4,16101.0,2017-04-27 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,0.010397,0.010672,0.010672,0.012819,0.192822,0.112189,...,36526.0,425.0,65.16,1591.366,0.0,273.0,"Yes. Well, first, thanks for the question. May...",Positive,"[0.26715648, 0.11978915, 0.61305434]","[Negative, Positive, Positive, Positive, Posit..."


In [18]:
# create function to calcualte sentiment score
def calculate_sentiment_score_BERT(probs):
    return probs[2] - probs[0]  # Positive probability minus Negative probability

In [19]:
# calcualte sentiment score for presentations
EC_pres_llm['sentiment_score_pres_finbert'] = EC_pres_llm["final_probabilities_pres_finbert"].apply(calculate_sentiment_score_BERT)

# define the scaler for standardization
pres_finbert_scaler = StandardScaler()

# standardize the sentiment score
EC_pres_llm['sentiment_score_pres_finbert_std'] = pres_finbert_scaler.fit_transform(EC_pres_llm[['sentiment_score_pres_finbert']])
EC_pres_llm.head()

,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,prccq,cshoq,dvpq,xintq,presentation,final_sentiment_pres_finbert,final_probabilities_pres_finbert,chunk_labels_pres_finbert,sentiment_score_pres_finbert,sentiment_score_pres_finbert_std
0,143356.0,2016-05-05 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.015353,-0.012682,-0.012682,-0.007504,0.259517,0.180580,...,72.22,443.146,0.0,325.0,"Ladies and gentlemen, thank you for standi...",Neutral,"[0.37680644, 0.50930405, 0.113889486]","[Neutral, Negative, Negative, Negative, Neutra...",-0.262917,0.477267
1,143356.0,2016-08-04 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.000416,-0.000815,-0.000815,-0.017714,0.239821,0.421283,...,71.34,438.739,0.0,357.0,Welcome to the Prudential quarterly earnin...,Negative,"[0.45286137, 0.33879948, 0.20833908]","[Positive, Negative, Negative, Negative, Neutr...",-0.244522,0.548455
2,143356.0,2016-11-03 13:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.031476,0.030910,0.030910,0.024372,0.277495,0.228244,...,81.65,431.704,0.0,314.0,"Ladies and gentlemen, thank you for standi...",Negative,"[0.59855324, 0.21789162, 0.18355516]","[Negative, Negative, Negative, Negative, Negat...",-0.414998,-0.111290
3,6730.0,2016-04-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.004689,-0.003578,-0.003578,-0.003950,0.213235,0.190130,...,72.01,1103.837,0.0,56.8,"Ladies and gentlemen, thank you for standi...",Negative,"[0.43463686, 0.30363905, 0.26172405]","[Positive, Negative, Negative, Positive, Posit...",-0.172913,0.825585
4,6730.0,2016-07-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.027560,0.027598,0.027598,0.021945,0.186548,0.109124,...,78.75,1103.843,0.0,57.2,"Ladies and gentlemen, thank you for standi...",Negative,"[0.6555745, 0.10334945, 0.24107622]","[Positive, Negative, Negative, Negative, Negat...",-0.414498,-0.109356


In [20]:
# calcualte sentiment score for answers
answers_llm['sentiment_score_answers_finbert'] = answers_llm["final_probabilities_answers_finbert"].apply(calculate_sentiment_score_BERT)

# define the scaler for standardization
answers_finbert_scaler = StandardScaler()

# standardize the sentiment score
answers_llm['sentiment_score_answers_finbert_std'] = answers_finbert_scaler.fit_transform(answers_llm[['sentiment_score_answers_finbert']])
answers_llm.head()

,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,prccq,cshoq,dvpq,xintq,QA_text,final_sentiment_answers_finbert,final_probabilities_answers_finbert,chunk_labels_answers_finbert,sentiment_score_answers_finbert,sentiment_score_answers_finbert_std
0,16101.0,2016-07-29 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.011886,0.014261,0.014261,0.021246,0.179151,0.129186,...,61.91,1628.542,0.0,245.0,"Jeff, this is Rick. I will cover the first and...",Positive,"[0.35596988, 0.05079206, 0.593238]","[Negative, Positive, Positive, Negative, Negat...",0.237268,-0.244995
1,16101.0,2016-04-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.026387,0.023499,0.023499,0.021770,0.289777,0.114447,...,57.12,1617.359,0.0,215.0,"Okay. Hi, Jeff, it's Rick. So I'll take I gues...",Positive,"[0.21798027, 0.027822783, 0.7541969]","[Positive, Positive, Negative, Positive, Posit...",0.536217,1.094665
2,16101.0,2016-10-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.078668,-0.079290,-0.079290,-0.092594,0.253269,0.381002,...,63.07,1624.908,0.0,271.0,"Sure, Jami; this is Rick. Thank you for the qu...",Positive,"[0.28801477, 0.061897594, 0.6500877]","[Positive, Positive, Positive, Negative, Posit...",0.362073,0.314286
3,16101.0,2017-01-27 14:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,-0.010152,-0.000737,-0.000737,-0.005279,0.182080,0.145941,...,62.62,1592.513,0.0,277.0,"Hi, Jami; it's Bill. So on your operating marg...",Positive,"[0.22084142, 0.030022986, 0.74913555]","[Negative, Positive, Positive, Positive, Negat...",0.528294,1.059163
4,16101.0,2017-04-27 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,0.010397,0.010672,0.010672,0.012819,0.192822,0.112189,...,65.16,1591.366,0.0,273.0,"Yes. Well, first, thanks for the question. May...",Positive,"[0.26715648, 0.11978915, 0.61305434]","[Negative, Positive, Positive, Positive, Posit...",0.345898,0.241801


In [21]:
# Convert the 'chunk_labels_pres_finbert' column to strings
EC_pres_llm['chunk_labels_pres_finbert'] = EC_pres_llm['chunk_labels_pres_finbert'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)

# Convert the 'chunk_labels_answers_finbert' column to strings
answers_llm['chunk_labels_answers_finbert'] = answers_llm['chunk_labels_answers_finbert'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)

# Now save to SQL
EC_pres_llm.to_sql('EC_pres_llm_finbert', conn, if_exists='replace', index=False)
answers_llm.to_sql('answers_llm_finbert', conn, if_exists='replace', index=False)

2877

**Approach 2: Train DistillBert on surject-finance-instsutrct 177k which contanies 44,209 entries for financial sentiment anlaysis, fine-tune tohe model, and use it to predict the earning calls**

**Model: https://huggingface.co/TheBloke/finance-LLM-GGUF**
** Training Data: https://huggingface.co/datasets/sujet-ai/Sujet-Finance-Instruct-177k**

**Goal: Do sentiment analysis using the distillBert trained for financial sentiment**

**Steps:**

*   Step 1: Import needed packages and load the Sujet-Finance_Instruct_177K dataset
*   Step 2: Insepect the dataset
*   Step 3:  Train the distillbert model
*   Step 4: Use the trained model for predicting the sentiment on the earning calls

**Step 1: Import needed packages and load the dataset**

In [22]:
# import packages
from datasets import load_dataset

# Load dataset
dataset = load_dataset("sujet-ai/Sujet-Finance-Instruct-177k")["train"]

In [23]:
# print the data
print(dataset)  # Overview of the Dataset
print(f"Number of rows: {len(dataset)}")  # Total rows
print(f"Features: {dataset.features}")  # Column names and types

Dataset({
    features: ['Unnamed: 0', 'inputs', 'answer', 'system_prompt', 'user_prompt', 'task_type', 'dataset', 'index_level', 'conversation_id'],
    num_rows: 177597
})
Number of rows: 177597
Features: {'Unnamed: 0': Value(dtype='int64', id=None), 'inputs': Value(dtype='string', id=None), 'answer': Value(dtype='string', id=None), 'system_prompt': Value(dtype='string', id=None), 'user_prompt': Value(dtype='string', id=None), 'task_type': Value(dtype='string', id=None), 'dataset': Value(dtype='string', id=None), 'index_level': Value(dtype='float64', id=None), 'conversation_id': Value(dtype='float64', id=None)}


In [24]:
# filter for the sentiment
sentiment_dataset = dataset.filter(lambda x: x["task_type"] == "sentiment_analysis")
print(sentiment_dataset)  # Overview of the filtered dataset
print(f"Number of sentiment analysis rows: {len(sentiment_dataset)}")  # Rows with sentiment analysis

Dataset({
    features: ['Unnamed: 0', 'inputs', 'answer', 'system_prompt', 'user_prompt', 'task_type', 'dataset', 'index_level', 'conversation_id'],
    num_rows: 44209
})
Number of sentiment analysis rows: 44209


In [25]:
# Check unique labels
unique_labels = set(sentiment_dataset["answer"])
print("Unique labels in the dataset:", unique_labels)

Unique labels in the dataset: {'Neutral', 'bearish', 'negative', 'bullish', 'neutral', 'Negative', 'positive', 'Positive'}


In [26]:
# take a look at a few examples
print("First 5 examples with labels:")
for i in range(5):
    print(f"Example {i}:")
    print(f"Text: {sentiment_dataset[i]['user_prompt']}")
    print(f"Label: {sentiment_dataset[i]['answer']}\n")

First 5 examples with labels:
Example 0:
Text: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Label: neutral

Example 1:
Text: Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .
Label: neutral

Example 2:
Text: The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .
Label: negative

Example 3:
Text: With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .
Label: positive

Example 4:
Text: According to the company 's updated strategy for the years 2009-

**Step 3: train the distilbert-base-uncased model**


In [27]:
from transformers import AutoTokenizer
# tokenize and label
tokenizer_distilbert = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize
def tokenize_function(examples):
    return tokenizer(examples["user_prompt"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = sentiment_dataset.map(tokenize_function, batched=True)

# Normalize labels and map
label_map = {"negative": 0, "bearish": 0, "neutral": 1, "positive": 2, "bullish": 2}
tokenized_dataset = tokenized_dataset.map(lambda x: {"labels": label_map[x["answer"].lower()]})
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [28]:
# Print a few tokenized examples to confirm
for i in range(5):
    print(f"Example {i}:")
    print(f"Text: {sentiment_dataset[i]['user_prompt']}")
    print(f"Original label: {sentiment_dataset[i]['answer']}")
    print(f"Mapped label: {tokenized_dataset[i]['labels']}\n")

Example 0:
Text: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
Original label: neutral
Mapped label: 1

Example 1:
Text: Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .
Original label: neutral
Mapped label: 1

Example 2:
Text: The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .
Original label: negative
Mapped label: 0

Example 3:
Text: With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .
Original label: positive
Mapped label: 2

Example 4:


In [29]:
# split into train/test
#train_size    = int(0.8 * len(tokenized_dataset))
#train_dataset = tokenized_dataset.shuffle().select(range(train_size))
#eval_dataset  = tokenized_dataset.shuffle().select(range(train_size, len(tokenized_dataset)))

In [30]:
# train the model
#from transformers import AutoTokenizer, AutoModelForSequenceClassification,Trainer, TrainingArguments

#model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)
#training_args = TrainingArguments(
 #   output_dir="./finetuned_distilbert",
 #   num_train_epochs=3,
 #   per_device_train_batch_size=8,
 #   evaluation_strategy="epoch",
 #   save_strategy="epoch",
 #   load_best_model_at_end=True,
 #   run_name="sentiment_finetune_sujet_177k",
#)
#trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset)
#trainer.train()
#trainer.save_model("finetuned_distilbert")
#tokenizer.save_pretrained("finetuned_distilbert")

In [31]:
# verify that the model was saved
#!ls -lh ./finetuned_distilbert

In [32]:
# save to google drive to be sure
#from google.colab import drive
#drive.mount('/content/drive')

#!cp -r ./finetuned_distilbert /content/drive/MyDrive/finetuned_distilbert

**Step 4: Use the trained model for predicting the sentiment on the earning calls**

In [33]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Specify the path to the finetuned_distilbert directory on Google Drive
# Adjust the path based on where you saved it (e.g., MyDrive/finetuned_distilbert)
model_dir = "/content/drive/MyDrive/finetuned_distilbert"

# Check if the directory exists and list its contents
if os.path.exists(model_dir):
    print(f"Model directory '{model_dir}' exists.")
    print("Contents:", os.listdir(model_dir))
else:
    print(f"Model directory '{model_dir}' does not exist.")
    print("Please check the path or re-fine-tune the model.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model directory '/content/drive/MyDrive/finetuned_distilbert' exists.
Contents: ['checkpoint-8842', 'runs', 'checkpoint-4421', 'checkpoint-13263', 'config.json', 'model.safetensors', 'training_args.bin', 'tokenizer_config.json', 'special_tokens_map.json', 'vocab.txt', 'tokenizer.json']


In [34]:
# load the model
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load tokenizer
try:
    distilbert_tokenizer = AutoTokenizer.from_pretrained(model_dir)
    print("Tokenizer loaded successfully.")
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    raise

# Load model
try:
    distilbert_model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    distilbert_model.to(device)
    print("Model loaded successfully and moved to device.")
except Exception as e:
    print(f"Error loading model: {e}")
    raise

# Verify model configuration
print("Model configuration:", distilbert_model.config)
num_labels = distilbert_model.config.num_labels
print(f"Number of labels: {num_labels}")
if num_labels != 3:
    print("Warning: Expected 3 labels (Negative, Neutral, Positive). Check fine-tuning process.")

Using device: cuda
Tokenizer loaded successfully.
Model loaded successfully and moved to device.
Model configuration: DistilBertConfig {
  "_attn_implementation_autoset": true,
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.50.3",
  "vocab_size": 30522
}

Number of labels: 3


In [35]:
# Process presentations
final_labels_pres_distilbert = []
probabilities_pres_distilbert = []
all_chunk_labels_pres_distilbert = []

for idx, row in tqdm(EC_pres_llm.iterrows(), total=EC_pres_llm.shape[0], desc="Processing presentations"):
    text = row["presentation"]
    final_label, final_probs, chunk_labels = analyze_long_text_sentiment(
        text,
        distilbert_tokenizer,
        distilbert_model,
        max_length=450,
        overlap_sentences=2,
        device=device
    )
    final_labels_pres_distilbert.append(final_label)
    probabilities_pres_distilbert.append(final_probs)
    all_chunk_labels_pres_distilbert.append(chunk_labels)

# Add to DataFrame
EC_pres_llm["final_sentiment_pres_distilbert"] = final_labels_pres_distilbert
EC_pres_llm["final_probabilities_pres_distilbert"] = probabilities_pres_distilbert
EC_pres_llm["chunk_labels_pres_distilbert"] = all_chunk_labels_pres_distilbert

Processing presentations: 100%|██████████| 2877/2877 [05:31<00:00,  8.68it/s]


In [36]:
# Process answers
final_labels_answers_distilbert = []
probabilities_answers_distilbert = []
all_chunk_labels_answers_distilbert = []

for idx, row in tqdm(answers_llm.iterrows(), total=answers_llm.shape[0], desc="Processing answers"):
    text = row["QA_text"]
    final_label, final_probs, chunk_labels = analyze_long_text_sentiment(
        text,
        distilbert_tokenizer,
        distilbert_model,
        max_length=450,
        overlap_sentences=2,
        device=device
    )
    final_labels_answers_distilbert.append(final_label)
    probabilities_answers_distilbert.append(final_probs)
    all_chunk_labels_answers_distilbert.append(chunk_labels)

# Add to DataFrame
answers_llm["final_sentiment_answers_distilbert"] = final_labels_answers_distilbert
answers_llm["final_probabilities_answers_distilbert"] = probabilities_answers_distilbert
answers_llm["chunk_labels_answers_distilbert"] = all_chunk_labels_answers_distilbert

Processing answers: 100%|██████████| 2877/2877 [06:46<00:00,  7.08it/s]


In [37]:
# Presentations
EC_pres_llm['sentiment_score_pres_distilbert'] = EC_pres_llm["final_probabilities_pres_distilbert"].apply(calculate_sentiment_score_BERT)
pres_distilbert_scaler = StandardScaler()
EC_pres_llm['sentiment_score_pres_distilbert_std'] = pres_distilbert_scaler.fit_transform(EC_pres_llm[['sentiment_score_pres_distilbert']])

# Answers
answers_llm['sentiment_score_answers_distilbert'] = answers_llm["final_probabilities_answers_distilbert"].apply(calculate_sentiment_score_BERT)
answers_distilbert_scaler = StandardScaler()
answers_llm['sentiment_score_answers_distilbert_std'] = answers_distilbert_scaler.fit_transform(answers_llm[['sentiment_score_answers_distilbert']])

In [38]:
# Convert the 'chunk_labels_pres_distilbert' column to strings
EC_pres_llm['chunk_labels_pres_distilbert'] = EC_pres_llm['chunk_labels_pres_distilbert'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)

# Convert the 'chunk_labels_answers_distilbert' column to strings
answers_llm['chunk_labels_answers_distilbert'] = answers_llm['chunk_labels_answers_distilbert'].apply(lambda x: ','.join(x) if isinstance(x, list) else x)

# save the results to sql
EC_pres_llm.to_sql('EC_pres_llm_finbert_distil', conn, if_exists='replace', index=False)
answers_llm.to_sql('answers_llm_finbert_distil', conn, if_exists='replace', index=False)

2877

**Approach 3: Use chat gpt to compare with FinBert**

**Model: https://huggingface.co/TheBloke/finance-LLM-GGUF**

**Goal: Do sentiment analysis using chat gpt api model 3.5 turbo to check is better than FinBert at assessing sentiment*

**Steps:**

*   Step 1: Import needed packages and initalize the model
*   Step 2 Anonymize the company names
*   Step 3: Performe chunking by sentences and uses overlapping  
to make sure that  does not classify based on not full sentences.
*   Step 4:Performe sentiment scoring


**Step 1: Import needed packages and initalize the model**

In [39]:
%%capture
!pip install --quiet --upgrade openai

In [40]:
!pip install tiktoken

In [41]:
import openai
import os
import tiktoken

# Get API key from Colab secrets
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

# Configure OpenAI client (new style)
client = openai.OpenAI(api_key=api_key)

# load tokeinzer for chunking
tokenizer_chatgpt = tiktoken.get_encoding("cl100k_base")

# Load spaCy for anonymization
nlp = spacy.load('en_core_web_sm')

**Step 2: Anonymize the company names**

In [42]:
# Anonymize the text
def anonymize_text(text):
    doc = nlp(text)
    anonymized_text = text
    for ent in doc.ents:
        if ent.label_ == 'ORG':
            anonymized_text = anonymized_text.replace(ent.text, '[COMPANY]')
    return anonymized_text

In [43]:
# anonymize the text
EC_pres_llm['presentation_anonym']      = EC_pres_llm['presentation'].progress_apply(anonymize_text)
answers_llm['QA_text_anonynm']          = answers_llm['QA_text'].progress_apply(anonymize_text)

100%|██████████| 2877/2877 [29:25<00:00,  1.63it/s]


In [44]:
# Count how many rows contain the placeholder in each column
pres_contains_company = EC_pres_llm['presentation_anonym'].str.contains(r'\[COMPANY\]').sum()
qa_contains_company   = answers_llm['QA_text_anonynm'].str.contains(r'\[COMPANY\]').sum()

print(f"Presentations with [COMPANY]: {pres_contains_company} out of {len(EC_pres_llm)}")
print(f"QA texts with [COMPANY]: {qa_contains_company} out of {len(answers_llm)}")


Presentations with [COMPANY]: 2876 out of 2877
QA texts with [COMPANY]: 2876 out of 2877


In [45]:
# Select a random sample of 10 rows where [COMPANY] appears in the 'presentation' column
sample_pres = EC_pres_llm[EC_pres_llm['presentation_anonym'].str.contains(r'\[COMPANY\]')].sample(1, random_state=42)
print("Sample anonymized presentations:")
print(sample_pres[['presentation_anonym']].to_string(index=False))

# Do the same for the QA texts
sample_qa = answers_llm[answers_llm['QA_text_anonynm'].str.contains(r'\[COMPANY\]')].sample(1, random_state=42)
print("\nSample anonymized QA texts:")
print(sample_qa[['QA_text_anonynm']].to_string(index=False))


Sample anonymized presentations:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

**Step 3: Create prompt function to score sentiment.**

In [51]:
# import json
import json

# create function to prompt chatgpt
def openai_llm_sentiment(text, client, section="Presentation", model="gpt-3.5-turbo", max_tokens=200, temperature=0.0):
    system_msg = (
        "You are a financial analyst specializing in sentiment analysis of earnings call transcripts. "
        "Return only a valid JSON object with no extra text, markdown, or explanations."
    )
    prompt = (
        f"Analyze the sentiment of this earnings call {section.lower()} section. "
        "Classify it as 'Negative', 'Neutral', or 'Positive'. Provide probabilities for each class "
        "(Negative, Neutral, Positive) as a list of 3 numbers summing to 1 (e.g., [0.1, 0.3, 0.6]). "
        "Return only a JSON object: {\"label\": \"Sentiment\", \"probabilities\": [neg, neu, pos]}. "
        "Example: {\"label\": \"Positive\", \"probabilities\": [0.1, 0.2, 0.7]}\n\n"
        f"{section} Transcript: {text}"
    )

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": system_msg}, {"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=temperature
    )

    result = response.choices[0].message.content.strip()
    #print(f"Raw ChatGPT Response: {result}")

    try:
        result_dict = json.loads(result)
        label = result_dict["label"]
        probs = result_dict["probabilities"]
        if len(probs) != 3 or abs(sum(probs) - 1.0) > 0.01:
            raise ValueError("Invalid probabilities")
    except Exception as e:
        print(f"JSON Parse Error: {e}, Response: {result}")
        label = "Neutral"
        probs = [0.33, 0.34, 0.33]

    return label, probs, response.usage.prompt_tokens, response.usage.completion_tokens

In [52]:
# function to chunk text using chatgpt
def chunk_text_by_sentences_with_overlap_chatgpt(text, tokenizer, max_length=13000, overlap_sentences=2, metadata=None):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    token_count = 0

    for sent in sentences:
        sent_tokens = len(tokenizer.encode(sent))
        if token_count + sent_tokens > max_length:
            chunk = "".join(current_chunk)
            if metadata:
                chunk = f"Date: {metadata.get('date', 'Unknown')}, Section: {metadata.get('section', 'Unknown')}\n" + chunk
            chunks.append(chunk)
            current_chunk = current_chunk[-overlap_sentences:] if len(current_chunk) >= overlap_sentences else current_chunk
            token_count = sum(len(tokenizer.encode(s)) for s in current_chunk)
        current_chunk.append(sent)
        token_count += sent_tokens

    if current_chunk:
        chunk = "".join(current_chunk)
        if metadata:
            chunk = f"Date: {metadata.get('date', 'Unknown')}, Section: {metadata.get('section', 'Unknown')}\n" + chunk
        chunks.append(chunk)

    return chunks

In [53]:
def analyze_long_text_sentiment_chatgpt(text, client, tokenizer, section="Presentation", date=None, max_length=13000):
    metadata = {"date": date or "Unknown", "section": section}
    chunks = chunk_text_by_sentences_with_overlap_chatgpt(text, tokenizer, max_length=max_length, metadata=metadata)

    total_input_tokens = 0
    total_output_tokens = 0
    all_probs = []
    all_weights = []
    chunk_labels = []

    for chunk in chunks:
        label, probs, input_tokens, output_tokens = openai_llm_sentiment(chunk, client, section=section)
        total_input_tokens += input_tokens
        total_output_tokens += output_tokens
        token_count = len(tokenizer.encode(chunk))
        chunk_labels.append(label)
        all_probs.append(probs)
        all_weights.append(token_count)

    if all_weights:
        weighted_sum = np.zeros(3)
        total_weight = sum(all_weights)
        for prob, weight in zip(all_probs, all_weights):
            weighted_sum += np.array(prob) * weight
        final_probs = (weighted_sum / total_weight).tolist()
        final_label = ["Negative", "Neutral", "Positive"][np.argmax(final_probs)]
    else:
        final_label = "Neutral"
        final_probs = [0.33, 0.34, 0.33]

    return final_label, final_probs, chunk_labels, total_input_tokens, total_output_tokens, len(chunks)

**Step 4:Performe sentiment scoring**

In [54]:
# Process presentations
total_input_tokens_pres = 0
total_output_tokens_pres = 0
total_chunks_pres = 0
final_labels_pres_chatgpt = []
probabilities_pres_chatgpt = []
all_chunk_labels_pres_chatgpt = []

for idx, row in tqdm(EC_pres_llm.iterrows(), total=EC_pres_llm.shape[0], desc="Processing presentations"):
    text = row["presentation_anonym"]
    date = str(row.get("date_rdq", "Unknown"))
    final_label, final_probs, chunk_labels, input_tokens, output_tokens, chunk_count = analyze_long_text_sentiment_chatgpt(
        text, client, tokenizer_chatgpt, section="Presentation", date=date
    )
    final_labels_pres_chatgpt.append(final_label)
    probabilities_pres_chatgpt.append(final_probs)
    all_chunk_labels_pres_chatgpt.append(chunk_labels)
    total_input_tokens_pres += input_tokens
    total_output_tokens_pres += output_tokens
    total_chunks_pres += chunk_count

EC_pres_llm["final_sentiment_pres_chatgpt"] = final_labels_pres_chatgpt
EC_pres_llm["final_probabilities_pres_chatgpt"] = probabilities_pres_chatgpt
EC_pres_llm["chunk_labels_pres_chatgpt"] = all_chunk_labels_pres_chatgpt

# display
EC_pres_llm.head()


Processing presentations: 100%|██████████| 2877/2877 [35:06<00:00,  1.37it/s]


,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,sentiment_score_pres_finbert_std,final_sentiment_pres_distilbert,final_probabilities_pres_distilbert,chunk_labels_pres_distilbert,sentiment_score_pres_distilbert,sentiment_score_pres_distilbert_std,presentation_anonym,final_sentiment_pres_chatgpt,final_probabilities_pres_chatgpt,chunk_labels_pres_chatgpt
0,143356.0,2016-05-05 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.015353,-0.012682,-0.012682,-0.007504,0.259517,0.180580,...,0.477267,Negative,"[0.5381766, 0.0007886944, 0.46103472]","Positive,Negative,Negative,Negative,Positive,P...",-0.077142,1.711852,"Ladies and gentlemen, thank you for standi...",Negative,"[0.8, 0.15, 0.05]",[Negative]
1,143356.0,2016-08-04 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.000416,-0.000815,-0.000815,-0.017714,0.239821,0.421283,...,0.548455,Negative,"[0.5613208, 0.06873317, 0.36994597]","Negative,Negative,Negative,Negative,Positive,P...",-0.191375,1.303720,Welcome to the [COMPANY] quarterly earning...,Positive,"[0.1, 0.1, 0.8]",[Positive]
2,143356.0,2016-11-03 13:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.031476,0.030910,0.030910,0.024372,0.277495,0.228244,...,-0.111290,Negative,"[0.86557025, 0.05048115, 0.08394858]","Negative,Negative,Negative,Negative,Negative,P...",-0.781622,-0.805119,"Ladies and gentlemen, thank you for standi...",Positive,"[0.1, 0.2, 0.7]",[Positive]
3,6730.0,2016-04-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.004689,-0.003578,-0.003578,-0.003950,0.213235,0.190130,...,0.825585,Negative,"[0.4297344, 0.18014105, 0.39012456]","Negative,Negative,Positive,Positive,Positive,P...",-0.039610,1.845947,"Ladies and gentlemen, thank you for standi...",Positive,"[0.1, 0.2, 0.7]",[Positive]
4,6730.0,2016-07-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.027560,0.027598,0.027598,0.021945,0.186548,0.109124,...,-0.109356,Negative,"[0.6420919, 0.17303307, 0.18487516]","Neutral,Negative,Negative,Negative,Negative,Po...",-0.457217,0.353917,"Ladies and gentlemen, thank you for standi...",Negative,"[0.9, 0.05, 0.05]",[Positive]


In [56]:
# Convert list columns to strings before saving to SQL
for column in ['final_probabilities_pres_finbert', 'chunk_labels_pres_finbert',
              'final_probabilities_pres_distilbert', 'chunk_labels_pres_distilbert',
              'final_probabilities_pres_chatgpt', 'chunk_labels_pres_chatgpt']:
    EC_pres_llm[column] = EC_pres_llm[column].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)


# save to sql
EC_pres_llm.to_sql('EC_pres_llm_FinB_Distil_chatgpt', conn, if_exists='replace', index=False)

2877

In [57]:
# Print final results
print("\n=== Final Results ===")
print(f"Final Label: {final_labels_pres_chatgpt}")
print(f"Final Probabilities (Negative, Neutral, Positive): {probabilities_pres_chatgpt}")
print(f"Chunk Labels: {all_chunk_labels_pres_chatgpt}")


=== Final Results ===
Final Label: ['Negative', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Negative', 'Positive', 'Positive', 'Negative', 'Negative', 'Negative', 'Positive', 'Neutral', 'Positive', 'Positive', 'Positive', 'Negative', 'Neutral', 'Neutral', 'Neutral', 'Positive', 'Neutral', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Negative', 'Positive', 'Positive', 'Neutral', 'Negative', 'Negative', 'Positive', 'Neutral', 'Negative', 'Positive', 'Negative', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive', 'Negative', 'Negative', 'Positive', 'Negative', 'Positive', 'Positive', 'Negative', 'Positive', 'Positive', 'Negative', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Neutral', 'Negative', 'Positive', 'Positive', 'Positive', 'Neutral', 'Neutral', 'Positive', 'Positive', 'Negative', '

In [58]:
# Process answers
total_input_tokens_ans = 0
total_output_tokens_ans = 0
total_chunks_ans = 0
final_labels_answers_chatgpt = []
probabilities_answers_chatgpt = []
all_chunk_labels_answers_chatgpt = []

for idx, row in tqdm(answers_llm.iterrows(), total=answers_llm.shape[0], desc="Processing answers"):
    text = row["QA_text_anonynm"]
    date = str(row.get("date_rdq", "Unknown"))
    final_label, final_probs, chunk_labels, input_tokens, output_tokens, chunk_count = analyze_long_text_sentiment_chatgpt(
        text, client, tokenizer_chatgpt, section="Q&A", date=date
    )
    final_labels_answers_chatgpt.append(final_label)
    probabilities_answers_chatgpt.append(final_probs)
    all_chunk_labels_answers_chatgpt.append(chunk_labels)
    total_input_tokens_ans += input_tokens
    total_output_tokens_ans += output_tokens
    total_chunks_ans += chunk_count

answers_llm["final_sentiment_answers_chatgpt"] = final_labels_answers_chatgpt
answers_llm["final_probabilities_answers_chatgpt"] = probabilities_answers_chatgpt
answers_llm["chunk_labels_answers_chatgpt"] = all_chunk_labels_answers_chatgpt

# display
answers_llm.head()

Processing answers: 100%|██████████| 2877/2877 [39:17<00:00,  1.22it/s]


,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,sentiment_score_answers_finbert_std,final_sentiment_answers_distilbert,final_probabilities_answers_distilbert,chunk_labels_answers_distilbert,sentiment_score_answers_distilbert,sentiment_score_answers_distilbert_std,QA_text_anonynm,final_sentiment_answers_chatgpt,final_probabilities_answers_chatgpt,chunk_labels_answers_chatgpt
0,16101.0,2016-07-29 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.011886,0.014261,0.014261,0.021246,0.179151,0.129186,...,-0.244995,Negative,"[0.62905973, 0.29857135, 0.07236887]","Negative,Neutral,Negative,Negative,Negative,Ne...",-0.556691,0.002307,"Jeff, this is Rick. I will cover the first and...",Negative,"[0.4, 0.4, 0.2]",[Neutral]
1,16101.0,2016-04-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.026387,0.023499,0.023499,0.021770,0.289777,0.114447,...,1.094665,Negative,"[0.48134628, 0.47063518, 0.04801858]","Negative,Negative,Negative,Neutral,Neutral,Neg...",-0.433328,0.555618,"Okay. Hi, Jeff, it's Rick. So I'll take I gues...",Negative,"[0.8, 0.1, 0.1]",[Positive]
2,16101.0,2016-10-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.078668,-0.079290,-0.079290,-0.092594,0.253269,0.381002,...,0.314286,Neutral,"[0.426294, 0.49842042, 0.07528567]","Neutral,Negative,Neutral,Negative,Neutral,Neut...",-0.351008,0.924838,"Sure, Jami; this is Rick. Thank you for the qu...",Negative,"[0.4, 0.4, 0.2]",[Neutral]
3,16101.0,2017-01-27 14:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,-0.010152,-0.000737,-0.000737,-0.005279,0.182080,0.145941,...,1.059163,Neutral,"[0.29378477, 0.50687236, 0.19934289]","Positive,Neutral,Neutral,Neutral,Positive,Posi...",-0.094442,2.075593,"Hi, Jami; it's Bill. So on your operating marg...",Negative,"[0.4, 0.4, 0.2]",[Neutral]
4,16101.0,2017-04-27 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,0.010397,0.010672,0.010672,0.012819,0.192822,0.112189,...,0.241801,Negative,"[0.54689384, 0.33489963, 0.11820647]","Negative,Negative,Negative,Neutral,Neutral,Neg...",-0.428687,0.576430,"Yes. Well, first, thanks for the question. May...",Negative,"[0.4, 0.4, 0.2]",[Neutral]


In [71]:
# Convert string columns back to lists for answers_llm
for column in ['final_probabilities_answers_finbert', 'chunk_labels_answers_finbert',
               'final_probabilities_answers_distilbert', 'chunk_labels_answers_distilbert',
               'final_probabilities_answers_chatgpt', 'chunk_labels_answers_chatgpt']:
    if 'probabilities' in column:
        answers_llm[column] = answers_llm[column].apply(lambda x: [float(i) for i in x.split(',')] if isinstance(x, str) and x else x)
    else:
        answers_llm[column] = answers_llm[column].apply(lambda x: x.split(',') if isinstance(x, str) and x else x)

# Convert string columns back to lists for EC_pres_llm
for column in ['final_probabilities_pres_chatgpt', 'chunk_labels_pres_chatgpt']:
    if 'probabilities' in column:
        EC_pres_llm[column] = EC_pres_llm[column].apply(lambda x: [float(i) for i in x.split(',')] if isinstance(x, str) and x else x)
    else:
        EC_pres_llm[column] = EC_pres_llm[column].apply(lambda x: x.split(',') if isinstance(x, str) and x else x)

In [72]:
# Print final results
print("\n=== Final Results ===")
print(f"Final Label: {final_labels_answers_chatgpt}")
print(f"Final Probabilities (Negative, Neutral, Positive): {probabilities_answers_chatgpt}")
print(f"Chunk Labels: {all_chunk_labels_answers_chatgpt}")


=== Final Results ===
Final Label: ['Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Positive', 'Negative', 'Negative', 'Positive', 'Negative', 'Negative', 'Negative', 'Positive', 'Negative', 'Negative', 'Negative', 'Positive', 'Negative', 'Negative', 'Negative', 'Negative', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive', 'Negative', 'Positive', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Neutral', 'Positive', 'Negative', 'Negative', 'Negative', 'Negative', 'Positive', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Negative', 'Positive', 'Negative', 'Negative', 'Negative', 'Positive', 'Negative', 'Negative', 'Negative', 'Negative', 'Positive', 'Negative', 'Negative', 'Negative', 'Positive', 'Positive', 'Negative', 'Neg

In [73]:
# Calculate totals
total_tokens = total_input_tokens_pres + total_output_tokens_pres + total_input_tokens_ans + total_output_tokens_ans
total_chunks = total_chunks_pres + total_chunks_ans
cost = (total_input_tokens_pres + total_input_tokens_ans) * 0.0005 / 1000 + \
       (total_output_tokens_pres + total_output_tokens_ans) * 0.0015 / 1000

print(f"\n=== Final Token and Cost Report ===")
print(f"Presentation: {total_input_tokens_pres} input, {total_output_tokens_pres} output, {total_chunks_pres} chunks")
print(f"Answers: {total_input_tokens_ans} input, {total_output_tokens_ans} output, {total_chunks_ans} chunks")
print(f"Total Tokens: {total_tokens}")
print(f"Total Chunks: {total_chunks}")
print(f"Total Cost: ${cost:.2f}")


=== Final Token and Cost Report ===
Presentation: 12275448 input, 74880 output, 2880 chunks
Answers: 14323996 input, 75062 output, 2887 chunks
Total Tokens: 26749386
Total Chunks: 5767
Total Cost: $13.52


In [74]:
# calcuclate and standardize the sentiment scores for presentations
EC_pres_llm['sentiment_score_pres_chatgpt'] = EC_pres_llm["final_probabilities_pres_chatgpt"].apply(calculate_sentiment_score_BERT)
pres_chatgpt_scaler = StandardScaler()
EC_pres_llm['sentiment_score_pres_chatgpt_std'] = pres_chatgpt_scaler.fit_transform(EC_pres_llm[['sentiment_score_pres_chatgpt']])

# calculate and standardize the sentiment scores for answers
answers_llm['sentiment_score_answers_chatgpt'] = answers_llm["final_probabilities_answers_chatgpt"].apply(calculate_sentiment_score_BERT)
answers_chatgpt_scaler = StandardScaler()
answers_llm['sentiment_score_answers_chatgpt_std'] = answers_chatgpt_scaler.fit_transform(answers_llm[['sentiment_score_answers_chatgpt']])

In [75]:
# take a look
EC_pres_llm

,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,final_probabilities_pres_distilbert,chunk_labels_pres_distilbert,sentiment_score_pres_distilbert,sentiment_score_pres_distilbert_std,presentation_anonym,final_sentiment_pres_chatgpt,final_probabilities_pres_chatgpt,chunk_labels_pres_chatgpt,sentiment_score_pres_chatgpt,sentiment_score_pres_chatgpt_std
0,143356.0,2016-05-05 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.015353,-0.012682,-0.012682,-0.007504,0.259517,0.180580,...,"[0.5381766, 0.0007886944, 0.46103472]","Positive,Negative,Negative,Negative,Positive,P...",-0.077142,1.711852,"Ladies and gentlemen, thank you for standi...",Negative,"[0.8, 0.15, 0.05]",[Negative],-0.75,-2.022895
1,143356.0,2016-08-04 15:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.000416,-0.000815,-0.000815,-0.017714,0.239821,0.421283,...,"[0.5613208, 0.06873317, 0.36994597]","Negative,Negative,Negative,Negative,Positive,P...",-0.191375,1.303720,Welcome to the [COMPANY] quarterly earning...,Positive,"[0.1, 0.1, 0.8]",[Positive],0.70,0.523486
2,143356.0,2016-11-03 13:00:00+00:00,PRUDENTIAL FINANCIAL INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.031476,0.030910,0.030910,0.024372,0.277495,0.228244,...,"[0.86557025, 0.05048115, 0.08394858]","Negative,Negative,Negative,Negative,Negative,P...",-0.781622,-0.805119,"Ladies and gentlemen, thank you for standi...",Positive,"[0.1, 0.2, 0.7]",[Positive],0.60,0.347873
3,6730.0,2016-04-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.004689,-0.003578,-0.003578,-0.003950,0.213235,0.190130,...,"[0.4297344, 0.18014105, 0.39012456]","Negative,Negative,Positive,Positive,Positive,P...",-0.039610,1.845947,"Ladies and gentlemen, thank you for standi...",Positive,"[0.1, 0.2, 0.7]",[Positive],0.60,0.347873
4,6730.0,2016-07-26 13:00:00+00:00,LILLY (ELI) & CO,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.027560,0.027598,0.027598,0.021945,0.186548,0.109124,...,"[0.6420919, 0.17303307, 0.18487516]","Neutral,Negative,Negative,Negative,Negative,Po...",-0.457217,0.353917,"Ladies and gentlemen, thank you for standi...",Negative,"[0.9, 0.05, 0.05]",[Positive],-0.85,-2.198508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2872,7906.0,2020-12-18 22:00:00+00:00,NIKE INC -CL B,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2020...,0.034369,0.035156,0.035156,0.026788,0.435623,0.230338,...,"[0.8767416, 0.12283013, 0.0004283458]","Neutral,Negative,Negative,Negative,Negative,Ne...",-0.876313,-1.143434,"Good afternoon, everyone. Welcome to [COMP...",Positive,"[0.05, 0.1, 0.8499999999999999]",[Positive],0.80,0.699098
2873,31846.0,2020-12-18 13:30:00+00:00,DARDEN RESTAURANTS INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2020...,0.010655,-0.001192,-0.001192,-0.008571,0.422331,0.295128,...,"[0.6201096, 0.15447924, 0.22541115]","Positive,Negative,Negative,Negative,Positive,N...",-0.394698,0.577283,"Ladies and gentlemen, thank you for standi...",Positive,"[0.1, 0.2, 0.7]",[Positive],0.60,0.347873
2874,6669.0,2020-12-17 16:00:00+00:00,LENNAR CORP,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2020...,NaN,NaN,NaN,NaN,NaN,NaN,...,"[0.7921019, 0.1383651, 0.06953302]","Neutral,Negative,Negative,Negative,Negative,Ne...",-0.722569,-0.594135,Welcome to [COMPANY]'s Fourth Quarter Earn...,Positive,"[0.049999999999999996, 0.09999999999999999, 0.85]",[Positive],0.80,0.699098
2875,3062.0,2020-12-22 15:00:00+00:00,CINTAS CORP,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2020...,-0.039536,-0.046764,-0.046764,-0.023906,0.291577,0.231272,...,"[0.8149834, 0.00039575822, 0.18462071]","Negative,Negative,Negative,Negative,Negative,P...",-0.630363,-0.264700,"Good day, everyone, and welcome to the [CO...",Positive,"[0.1, 0.2, 0.7]",[Positive],0.60,0.347873


In [76]:
# take a look
answers_llm

,GVKEY,date_rdq,co_conm,file_name,CAR-11-Carhart,CAR-11-ff3,CAR01-Carhart,CAR01-ff3,IV,hvol,...,final_probabilities_answers_distilbert,chunk_labels_answers_distilbert,sentiment_score_answers_distilbert,sentiment_score_answers_distilbert_std,QA_text_anonynm,final_sentiment_answers_chatgpt,final_probabilities_answers_chatgpt,chunk_labels_answers_chatgpt,sentiment_score_answers_chatgpt,sentiment_score_answers_chatgpt_std
0,16101.0,2016-07-29 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.011886,0.014261,0.014261,0.021246,0.179151,0.129186,...,"[0.62905973, 0.29857135, 0.07236887]","[Negative, Neutral, Negative, Negative, Negati...",-0.556691,0.002307,"Jeff, this is Rick. I will cover the first and...",Negative,"[0.4, 0.4, 0.2]",[Neutral],-0.20,-0.345943
1,16101.0,2016-04-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,0.026387,0.023499,0.023499,0.021770,0.289777,0.114447,...,"[0.48134628, 0.47063518, 0.04801858]","[Negative, Negative, Negative, Neutral, Neutra...",-0.433328,0.555618,"Okay. Hi, Jeff, it's Rick. So I'll take I gues...",Negative,"[0.8, 0.1, 0.1]",[Positive],-0.70,-1.236753
2,16101.0,2016-10-28 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.078668,-0.079290,-0.079290,-0.092594,0.253269,0.381002,...,"[0.426294, 0.49842042, 0.07528567]","[Neutral, Negative, Neutral, Negative, Neutral...",-0.351008,0.924838,"Sure, Jami; this is Rick. Thank you for the qu...",Negative,"[0.4, 0.4, 0.2]",[Neutral],-0.20,-0.345943
3,16101.0,2017-01-27 14:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,-0.010152,-0.000737,-0.000737,-0.005279,0.182080,0.145941,...,"[0.29378477, 0.50687236, 0.19934289]","[Positive, Neutral, Neutral, Neutral, Positive...",-0.094442,2.075593,"Hi, Jami; it's Bill. So on your operating marg...",Negative,"[0.4, 0.4, 0.2]",[Neutral],-0.20,-0.345943
4,16101.0,2017-04-27 13:00:00+00:00,ABBVIE INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2017...,0.010397,0.010672,0.010672,0.012819,0.192822,0.112189,...,"[0.54689384, 0.33489963, 0.11820647]","[Negative, Negative, Negative, Neutral, Neutra...",-0.428687,0.576430,"Yes. Well, first, thanks for the question. May...",Negative,"[0.4, 0.4, 0.2]",[Neutral],-0.20,-0.345943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2872,189491.0,2016-11-01 13:00:00+00:00,XYLEM INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.022716,-0.023069,-0.023069,-0.015313,0.238038,0.176583,...,"[0.6140554, 0.14047264, 0.24547203]","[Positive, Negative, Negative, Negative, Negat...",-0.368583,0.846010,"Yes, well, thanks for the question, Jim. First...",Negative,"[0.4, 0.4, 0.2]",[Neutral],-0.20,-0.345943
2873,189491.0,2016-08-02 13:00:00+00:00,XYLEM INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2016...,-0.036201,-0.034737,-0.034737,-0.037363,0.201698,0.213624,...,"[0.88840127, 0.0026615958, 0.10893711]","[Negative, Negative, Negative, Negative, Posit...",-0.779464,-0.996878,"Sure, Deane. Let me just characterize what we ...",Positive,"[0.1, 0.1, 0.8]",[Positive],0.70,1.257516
2874,189491.0,2020-07-30 13:00:00+00:00,XYLEM INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2020...,-0.030116,-0.028020,-0.028020,-0.049431,0.325614,0.345711,...,"[0.56900084, 0.4287152, 0.002283889]","[Negative, Neutral, Negative, Neutral, Neutral...",-0.566717,-0.042662,"That is correct, Dean, on both counts.Sure. Ye...",Positive,"[0.1, 0.05, 0.85]",[Positive],0.75,1.346597
2875,189491.0,2018-05-01 13:00:00+00:00,XYLEM INC,Download ECC/SE/TRANSCRIPT/XMLStd/Archive/2018...,-0.066260,-0.064927,-0.064927,-0.058477,0.206359,0.215869,...,"[0.78206927, 0.21720979, 0.00072088547]","[Negative, Neutral, Negative, Negative, Negati...",-0.781348,-1.005329,"Yes, I'll go first here, Nate. And I think the...",Positive,"[0.1, 0.2, 0.7]",[Positive],0.60,1.079354


In [77]:
# save the dataframes incase of not being able to re-run
EC_pres_llm.to_csv("EC_pres_llm_with_chatgpt.csv", index=False)
answers_llm.to_csv("answers_llm_with_chatgpt.csv", index=False)

In [79]:
# store to SQL
# Convert list columns to strings in EC_pres_llm
for column in ['final_probabilities_pres_chatgpt', 'chunk_labels_pres_chatgpt']:
    if column in EC_pres_llm.columns:
        EC_pres_llm[column] = EC_pres_llm[column].apply(
            lambda x: ','.join(map(str, x)) if isinstance(x, list) else x
        )

# Convert list columns to strings in answers_llm
for column in ['final_probabilities_answers_finbert', 'chunk_labels_answers_finbert',
               'final_probabilities_answers_distilbert', 'chunk_labels_answers_distilbert',
               'final_probabilities_answers_chatgpt', 'chunk_labels_answers_chatgpt']:
    if column in answers_llm.columns:
        answers_llm[column] = answers_llm[column].apply(
            lambda x: ','.join(map(str, x)) if isinstance(x, list) else x
        )


EC_pres_llm.to_sql('EC_pres_llm_all', conn, if_exists='replace', index=False)
answers_llm.to_sql('answers_llm_all', conn, if_exists='replace', index=False)
conn.close()

In [92]:
# save the sql files to google drive¨
#import time
# Create a timestamped filename, e.g., LLMs_results_20250409-142500.db
#timestamp = time.strftime("%Y%m%d-%H%M%S")
#destination = f"/content/drive/My Drive/SQL_1/LLMs_results_{timestamp}.db"

#!cp /content/LLMs_results.db "{destination}"

cp: cannot create regular file '/content/drive/My Drive/SQL_1/LLMs_results_20250409-181443.db': No such file or directory
